In [1]:
import rail
from rail.estimation.algos.deepdisc import *
from rail.core.data import TableHandle, JsonHandle, ModelHandle
from rail.core.stage import RailStage
import torch.multiprocessing as mp

In [2]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [3]:
model_file = "./test_informer.pkl"
model_handle = DS.add_data("model", data=None, handle_class=ModelHandle, path=model_file)
test_file_for_chunks = "/home/shared/hsc/DC2/test_data/dataset_3/flattened_images_test_small.hdf5"
test_handle_for_chunks = DS.add_data("testing", data=None, handle_class=TableHandle, path=test_file_for_chunks)
metadatafile_with_chunks = "/home/shared/hsc/DC2/test_data/dataset_3/test_metadata_example.hdf5"
metadatahandle_with_chunks = DS.add_data("metadata", data=None, handle_class=Hdf5Handle, path=metadatafile_with_chunks)

cfgfile = "./configs/solo/solo_swin_DC2.py"

In [4]:
#MH = ModelHandle('model',path='./test_informer.pkl')
#data = MH.read()

In [5]:
deep_estimation_dict = dict(
    chunk_size=5,
    output_dir="./",
    cfgfile = cfgfile,
    zmin=0,
    zmax=5,
    nzbins=200,
    output_mode='default',
)

EstimatorWithChunks = DeepDiscPDFEstimatorWithChunking.make_stage(
    name="DeepDiscEstimatorWithChunks",
    model=model_file,
    **deep_estimation_dict,
)

In [ ]:
results_from_chunks = EstimatorWithChunks.estimate(test_handle_for_chunks, metadatahandle_with_chunks)

/home/g4merz/.conda/envs/ddrailnv/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /data/miniconda3/envs/opence-1.7/conda-bld/pytorch-base_1672876060819/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
res_ens = results_from_chunks.read()


In [ ]:
writeout = QPHandle('output',path='./output_DeepDiscEstimatorWithChunks.hdf5')
data = writeout.read()

In [ ]:
data.npdf

In [ ]:
data.ancil

In [8]:
zgrid = np.linspace(0,5,200)
data[7].pdf(zgrid)

array([[0.        , 0.00201005, 0.0040201 , 0.00603015, 0.0080402 ,
        0.01005025, 0.0120603 , 0.01407035, 0.0160804 , 0.01809045,
        0.0201005 , 0.02211055, 0.0241206 , 0.02613065, 0.0281407 ,
        0.03015075, 0.0321608 , 0.03417085, 0.0361809 , 0.03819095,
        0.04020101, 0.04221106, 0.04422111, 0.04623116, 0.04824121,
        0.05025126, 0.05226131, 0.05427136, 0.05628141, 0.05829146,
        0.06030151, 0.06231156, 0.06432161, 0.06633166, 0.06834171,
        0.07035176, 0.07236181, 0.07437186, 0.07638191, 0.07839196,
        0.08040201, 0.08241206, 0.08442211, 0.08643216, 0.08844221,
        0.09045226, 0.09246231, 0.09447236, 0.09648241, 0.09849246,
        0.10050251, 0.10251256, 0.10452261, 0.10653266, 0.10854271,
        0.11055276, 0.11256281, 0.11457286, 0.11658291, 0.11859296,
        0.12060302, 0.12261307, 0.12462312, 0.12663317, 0.12864322,
        0.13065327, 0.13266332, 0.13467337, 0.13668342, 0.13869347,
        0.14070352, 0.14271357, 0.14472362, 0.14

In [18]:
data.pdf(zgrid)[4]

array([0.        , 0.00201005, 0.0040201 , 0.00603015, 0.0080402 ,
       0.01005025, 0.0120603 , 0.01407035, 0.0160804 , 0.01809045,
       0.0201005 , 0.02211055, 0.0241206 , 0.02613065, 0.0281407 ,
       0.03015075, 0.0321608 , 0.03417085, 0.0361809 , 0.03819095,
       0.04020101, 0.04221106, 0.04422111, 0.04623116, 0.04824121,
       0.05025126, 0.05226131, 0.05427136, 0.05628141, 0.05829146,
       0.06030151, 0.06231156, 0.06432161, 0.06633166, 0.06834171,
       0.07035176, 0.07236181, 0.07437186, 0.07638191, 0.07839196,
       0.08040201, 0.08241206, 0.08442211, 0.08643216, 0.08844221,
       0.09045226, 0.09246231, 0.09447236, 0.09648241, 0.09849246,
       0.10050251, 0.10251256, 0.10452261, 0.10653266, 0.10854271,
       0.11055276, 0.11256281, 0.11457286, 0.11658291, 0.11859296,
       0.12060302, 0.12261307, 0.12462312, 0.12663317, 0.12864322,
       0.13065327, 0.13266332, 0.13467337, 0.13668342, 0.13869347,
       0.14070352, 0.14271357, 0.14472362, 0.14673367, 0.14874

In [8]:
res.an

NameError: name 'res' is not defined

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
x = np.linspace(0,3,100)
ax.plot(x, res_ens[0].pdf(x)[0])
ax.plot(x, res_ens[1].pdf(x)[0])